In [4]:
import json

# Load the JSON data from the file
file_path = '/workspaces/codespaces-jupyter/data/qjaPunsX.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Initialize a dictionary to hold the number of cards per list and the labels
list_summary = {}

# Iterate through each list in the data
for lst in data.get('lists', []):
    if lst['closed']:  # Skip archived lists
        continue
    
    list_id = lst['id']
    list_name = lst['name']
    list_summary[list_id] = {
        'name': list_name,
        'cards': 0,
        'labels': set()
    }

# Iterate through each card to count them in their respective lists and gather labels
for card in data.get('cards', []):
    if card['closed']:  # Skip archived cards
        continue
    
    list_id = card['idList']
    if list_id in list_summary:
        list_summary[list_id]['cards'] += 1
        # Collect labels from the card
        for label in card.get('labels', []):
            list_summary[list_id]['labels'].add(label['name'])

list_summary  # Output the summary of lists with number of cards and labels


{'64c0ed70c76f4cbf215fd0ad': {'name': 'Abidjan local data',
  'cards': 2,
  'labels': {'Abidjan'}},
 '64e33850695c26dbbe586422': {'name': 'ANRS', 'cards': 2, 'labels': set()},
 '638723c479805e007b606ac9': {'name': 'Identified in review/Application or initial request needs to be sent',
  'cards': 3,
  'labels': {'JHB', 'Narrow Consent', 'Study Ongoing', 'VIDA'}},
 '6387246d95be9b0051b584f0': {'name': '1st Follow up needs to be actioned',
  'cards': 1,
  'labels': {'Abidjan'}},
 '638724bf3a4d010070ba0d3d': {'name': '2nd Follow up needs to be actioned',
  'cards': 1,
  'labels': {'JHB',
   'Narrow Consent',
   'Sanofi',
   'WCR Bara Clinical Trials unit'}},
 '638724dcf35792011f9fb35a': {'name': '3rd Follow up needs to be actioned',
  'cards': 1,
  'labels': {'JHB', 'Narrow Consent', 'PHRU', 'boehringer-ingelheim'}},
 '638724e5bb60b30517fde104': {'name': '4th follow up: reach out on LinkedIn',
  'cards': 1,
  'labels': {'Abidjan'}},
 '638724ee422a6300fb360a65': {'name': '5th follow up: if 

In [5]:
import pandas as pd

# Create a DataFrame to hold the card names and their associated list names
cards_data = {
    "Card Name": [],
    "List Name": []
}

# Map list IDs to names
list_id_to_name = {lst['id']: lst['name'] for lst in data.get('lists', [])}

# Iterate through each card to get its name and the list it belongs to
for card in data.get('cards', []):
    if card['closed']:  # Skip archived cards
        continue
    
    list_id = card['idList']
    list_name = list_id_to_name.get(list_id, "Unknown List")  # Handle possible unknown list ID
    cards_data["Card Name"].append(card['name'])
    cards_data["List Name"].append(list_name)

# Convert the data into a DataFrame
cards_df = pd.DataFrame(cards_data)
cards_df.head()  # Display the first few entries


,Card Name,List Name
0,Abidjan socio-economic data,Abidjan local data
1,DIS_MSHP,Abidjan local data
2,ANRS ABIDJAN,ANRS
3,DATA from ANRS,ANRS
4,JHB_PHRU_1416,Units


In [6]:
# Adding a column to the DataFrame to include the overall bucket name which we'll assume is the 'desc' field from the board
overall_bucket_name = data.get('name', "Unknown Bucket")

# Adding this information to the DataFrame
cards_df['Overall Bucket Name'] = overall_bucket_name

# Counting how many cards link to 'Wits RHI'
wits_rhi_count = sum('Wits RHI' in labels for labels in list_summary.values() if isinstance(labels, set))

cards_df.head(), wits_rhi_count


(                     Card Name           List Name   Overall Bucket Name
 0  Abidjan socio-economic data  Abidjan local data  Data Acquisition RP2
 1                     DIS_MSHP  Abidjan local data  Data Acquisition RP2
 2                 ANRS ABIDJAN                ANRS  Data Acquisition RP2
 3               DATA from ANRS                ANRS  Data Acquisition RP2
 4                JHB_PHRU_1416               Units  Data Acquisition RP2,
 0)

In [7]:
# Correcting the approach to count how many cards have the "Wits RHI" label among their labels
wits_rhi_cards = sum(any(label['name'] == 'Wits RHI' for label in card.get('labels', [])) 
                     for card in data.get('cards', []) 
                     if not card['closed'])

wits_rhi_cards


9

In [8]:
# Extracting and listing the cards that have the "Wits RHI" label and the list (stage) they are in
wits_rhi_cards_data = {
    "Card Name": [],
    "List Name": []
}

# Iterate through each card to check for "Wits RHI" label and gather their names and list names
for card in data.get('cards', []):
    if card['closed']:  # Skip archived cards
        continue
    
    # Check if "Wits RHI" is among the labels
    if any(label['name'] == 'Wits RHI' for label in card.get('labels', [])):
        list_id = card['idList']
        list_name = list_id_to_name.get(list_id, "Unknown List")  # Resolve list name from ID
        wits_rhi_cards_data["Card Name"].append(card['name'])
        wits_rhi_cards_data["List Name"].append(list_name)

# Convert the data into a DataFrame for better visualization
wits_rhi_cards_df = pd.DataFrame(wits_rhi_cards_data)
wits_rhi_cards_df


,Card Name,List Name
0,JHB_WitsRHI_26_MIRA_Acceptability,Expressed interest in participating/Next steps...
1,JHB_MTN-SCHARP_012_CHARISMA,Next steps or application sent/now to follow up
2,JHB_WitsRHI_40_ASPIRE,Next steps or application sent/now to follow up
3,JHB_WitsRHI_011_ECHO,confirmed eligibility/start DTA Process
4,JHB_WitsRHI_014_FACTS001,confirmed eligibility/start DTA Process
5,JHB_SCHARP_006_HPTN082,Data transfered to UCT
6,JHB_WRHI_Soweto_40_HPTN052,Data currently unavailable/start data acquisit...
7,JHB_WitsRHI_49_RECOVERY_COVID19,Data currently unavailable/start data acquisit...
8,JHB_WRHI_39_HPTN084,Data currently unavailable/start data acquisit...


In [9]:
# Extracting and listing the cards that have the "Wits RHI" label and the list (stage) they are in
wits_rhi_cards_data = {
    "Card Name": [],
    "List Name": []
}

# Iterate through each card to check for "Wits RHI" label and gather their names and list names
for card in data.get('cards', []):
    if card['closed']:  # Skip archived cards
        continue
    
    # Check if "Wits RHI" is among the labels
    if any(label['name'] == 'Wits RHI' for label in card.get('labels', [])):
        list_id = card['idList']
        list_name = list_id_to_name.get(list_id, "Unknown List")  # Resolve list name from ID
        wits_rhi_cards_data["Card Name"].append(card['name'])
        wits_rhi_cards_data["List Name"].append(list_name)

# Convert the data into a DataFrame for better visualization
wits_rhi_cards_df = pd.DataFrame(wits_rhi_cards_data)
wits_rhi_cards_df


,Card Name,List Name
0,JHB_WitsRHI_26_MIRA_Acceptability,Expressed interest in participating/Next steps...
1,JHB_MTN-SCHARP_012_CHARISMA,Next steps or application sent/now to follow up
2,JHB_WitsRHI_40_ASPIRE,Next steps or application sent/now to follow up
3,JHB_WitsRHI_011_ECHO,confirmed eligibility/start DTA Process
4,JHB_WitsRHI_014_FACTS001,confirmed eligibility/start DTA Process
5,JHB_SCHARP_006_HPTN082,Data transfered to UCT
6,JHB_WRHI_Soweto_40_HPTN052,Data currently unavailable/start data acquisit...
7,JHB_WitsRHI_49_RECOVERY_COVID19,Data currently unavailable/start data acquisit...
8,JHB_WRHI_39_HPTN084,Data currently unavailable/start data acquisit...


In [10]:
# Extracting and listing the cards that have the specified labels
# (PHRU, Wits RHI, Right to Care, Ezintsha) and their list (stage) they are in
specified_labels = ["PHRU", "Wits RHI", "Right to Care", "Ezintsha"]
cards_by_label = {
    "PHRU": {"cards": [], "list_names": []},
    "Wits RHI": {"cards": [], "list_names": []},
    "Right to Care": {"cards": [], "list_names": []},
    "Ezintsha": {"cards": [], "list_names": []}
}

# Iterate through each card to check for specified labels and gather their names and list names
for card in data.get('cards', []):
    if card['closed']:  # Skip archived cards
        continue
    
    # Gather the list name
    list_id = card['idList']
    list_name = list_id_to_name.get(list_id, "Unknown List")  # Resolve list name from ID
    
    # Check for each of the specified labels
    for label in card.get('labels', []):
        label_name = label['name']
        if label_name in specified_labels:
            cards_by_label[label_name]["cards"].append(card['name'])
            cards_by_label[label_name]["list_names"].append(list_name)

# Convert the data into DataFrames for better visualization
cards_by_label_dfs = {label: pd.DataFrame(data) for label, data in cards_by_label.items()}

# Preview the data for one of the labels
cards_by_label_dfs['Wits RHI']


,cards,list_names
0,JHB_WitsRHI_26_MIRA_Acceptability,Expressed interest in participating/Next steps...
1,JHB_MTN-SCHARP_012_CHARISMA,Next steps or application sent/now to follow up
2,JHB_WitsRHI_40_ASPIRE,Next steps or application sent/now to follow up
3,JHB_WitsRHI_011_ECHO,confirmed eligibility/start DTA Process
4,JHB_WitsRHI_014_FACTS001,confirmed eligibility/start DTA Process
5,JHB_SCHARP_006_HPTN082,Data transfered to UCT
6,JHB_WRHI_Soweto_40_HPTN052,Data currently unavailable/start data acquisit...
7,JHB_WitsRHI_49_RECOVERY_COVID19,Data currently unavailable/start data acquisit...
8,JHB_WRHI_39_HPTN084,Data currently unavailable/start data acquisit...
